In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

%matplotlib inline

In [ ]:
!tree

.
├── 1-Data-exploration.ipynb
├── 2-Learning.ipynb
├── 2-Learning-Iquitos.ipynb
├── 2-Learning-SanJuan-Copy1.ipynb
├── 2-Learning-SanJuan.ipynb
├── 2-Median Submission.ipynb
├── data_processing.py
├── dengue_features_test.csv
├── dengue_features_train.csv
├── dengue_labels_train.csv
├── Iquitos-plaza.jpg
├── median_as_prediction.csv
├── optimizationOfModels.py
├── __pycache__
│   ├── data_processing.cpython-36.pyc
│   └── optimizationOfModels.cpython-36.pyc
├── README.md
├── SanJuan-ElasticNet_Iquitos-RandomForest.csv
├── San_Juan-Puerto_Rico.JPG
├── SanJuan-RandomForest_Iquitos-RandomForest.csv
├── SanJuan-RandomForest_Iquitos-RandomForest-lags-15-20.csv
├── SanJuan-RandomForest_Iquitos-RandomForest-shifted.csv
└── submission_format.csv

1 directory, 22 files


# 1. Data import and processing 

In [ ]:
from data_processing import preprocess_data,split_time_series,preprocess_data_with_PCA

lag_step_for_SanJuan = 15
lag_step_for_Iquitos = 20

sanJuan_data, iquitos_data = preprocess_data('dengue_features_train.csv',
                                    labels_path="dengue_labels_train.csv",
                                    lag_step_for_SanJuan = lag_step_for_SanJuan,
                                    lag_step_for_Iquitos = lag_step_for_Iquitos)


sanJuan_features = sanJuan_data.drop('total_cases', axis=1)
sanJuan_targets = sanJuan_data['total_cases']

iquitos_features = iquitos_data.drop('total_cases', axis=1)
iquitos_targets = iquitos_data['total_cases']

/home/louis/Documents/Doctorat/OnlineDatasets/dengue-drivendata/data_processing.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe[column + "_time - " + str(lag)] = dataframe[column].shift(lag)  # copy the previous value
/home/louis/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2852: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


# 2. Training 

In [ ]:

test_set_ratio = 0.3 

X_sj_train, X_sj_test, y_sj_train, y_sj_test = split_time_series(sanJuan_features, sanJuan_targets, test_set_ratio)
X_iq_train, X_iq_test, y_iq_train, y_iq_test = split_time_series(iquitos_features, iquitos_targets, test_set_ratio)


## 2.1 Optimization of parameters for San Juan

In [ ]:
from optimizationOfModels import find_best_params_ElasticNet, find_best_params_RandomForest

In [ ]:
parameters1_SanJuan = [10,20,50,100,200,300,500]
parameters2_SanJuan = [10,15,20,40,50,100]

param1_sj, param2_sj, scores__mean_absulute_error_sj = find_best_params_RandomForest(X_sj_train,
                                                                                 y_sj_train,
                                                                                 X_sj_test,
                                                                                 y_sj_test,
                                                                                 parameters1_SanJuan,
                                                                                 parameters2_SanJuan)

In [ ]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores__mean_absulute_error_sj, interpolation='nearest', cmap=plt.cm.hot_r)
plt.ylabel('alpha')
plt.xlabel('--')
plt.colorbar()
plt.yticks(np.arange(len(parameters1_SanJuan)), parameters1_SanJuan, rotation=45)
plt.xticks(np.arange(len(parameters2_SanJuan)), parameters2_SanJuan)
plt.title('Validation accuracy for San Juan')
plt.show()

## 2.2 Optimization of parameters for Iquitos

In [ ]:
parameters1_Iquitos = [20,50,100,200,300,400,500]
parameters2_Iquitos = [10,15,20,40]

param1_iq, param2_iq, scores__mean_absulute_error_iq = find_best_params_RandomForest(X_iq_train,
                                                                                 y_iq_train,
                                                                                 X_iq_test,
                                                                                 y_iq_test,
                                                                                 parameters1_Iquitos,
                                                                                 parameters2_Iquitos)

In [ ]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores__mean_absulute_error_iq, interpolation='nearest', cmap=plt.cm.hot_r)
plt.ylabel('n_estimators')
plt.xlabel('max_depth')
plt.colorbar()
plt.yticks(np.arange(len(parameters1_Iquitos)), parameters1_Iquitos, rotation=45)
plt.xticks(np.arange(len(parameters2_Iquitos)), parameters2_Iquitos)
plt.title('Validation accuracy for Iquitos')
plt.show()

## 2.3 Final evaluation of models

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

#regressor_sj = ElasticNet(alpha=param1_sj,
#                          l1_ratio=param2_sj)
regressor_sj = RandomForestRegressor(n_estimators=param1_sj,
                                     max_depth=param2_sj,
                                     criterion='mse',
                                     warm_start = True,
                                     random_state=0,
                                     n_jobs=8)

regressor_iq = RandomForestRegressor(n_estimators=param1_iq,
                                     max_depth=param2_iq,
                                     criterion='mse',
                                     warm_start = True,
                                     random_state=0,
                                     n_jobs=8)

In [ ]:
regressor_sj.fit(X_sj_train,y_sj_train)


In [ ]:
regressor_iq.fit(X_iq_train,y_iq_train)

In [ ]:
df_features_importance = pd.concat((pd.DataFrame(X_sj_train.columns, columns = ['variable']), 
           pd.DataFrame(regressor_sj.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

df_features_importance.plot.bar(x=df_features_importance['variable'])

In [ ]:
predictions_sj = regressor_sj.predict(X_sj_test)
predictions_iq = regressor_iq.predict(X_iq_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("For San Juan\n ")
print("mean_squared_error: {0}".format(mean_squared_error(y_sj_test, predictions_sj)))
print("mean_absolute_error: {0}".format(mean_absolute_error(y_sj_test, predictions_sj)))


df1 = pd.DataFrame({"predicted":predictions_sj,"actual":y_sj_test})

df1.plot()
plt.title('San Juan: predicted vs actual')
plt.xlabel('Time')
plt.legend()

In [ ]:
print("\nFor Iquitos\n ")
print("mean_squared_error: {0}".format(mean_squared_error(y_iq_test, predictions_iq)))
print("mean_absolute_error: {0}".format(mean_absolute_error(y_iq_test, predictions_iq)))

df2 = pd.DataFrame({"predicted":predictions_iq,"actual":y_iq_test})

df2.plot()
plt.title('Iquitos: predicted vs actual')
plt.xlabel('Time')

plt.legend()

In [ ]:
predictions_sj = regressor_sj.predict(sanJuan_features)
predictions_iq = regressor_iq.predict(iquitos_features)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("For San Juan\n ")
print("mean_squared_error: {0}".format(mean_squared_error(sanJuan_targets, predictions_sj)))
print("mean_absolute_error: {0}".format(mean_absolute_error(sanJuan_targets, predictions_sj)))

df1 = pd.DataFrame({"predicted":predictions_sj,"actual":sanJuan_targets})

df1.plot()
plt.title('San Juan: predicted vs actual')
plt.xlabel('Time')
plt.legend()

In [ ]:
print("\nFor Iquitos\n ")
print("mean_squared_error: {0}".format(mean_squared_error(iquitos_targets, predictions_iq)))
print("mean_absolute_error: {0}".format(mean_absolute_error(iquitos_targets, predictions_iq)))


df2 = pd.DataFrame({"predicted":predictions_iq,"actual":iquitos_targets})

df2.plot()
plt.title('Iquitos: predicted vs actual')
plt.xlabel('Time')

plt.legend()

# 3. Submission

In [ ]:
regressor_sj.fit(sanJuan_features,sanJuan_targets)

In [ ]:
regressor_iq.fit(iquitos_features,iquitos_targets)

In [ ]:
from data_processing import preprocess_data_with_PCA_for_test

sj_test , iq_test = preprocess_data('dengue_features_test.csv',
                                   lag_step_for_SanJuan = lag_step_for_SanJuan,
                                   lag_step_for_Iquitos = lag_step_for_Iquitos)





sj_predictions = regressor_sj.predict(sj_test).astype(int)
iq_predictions = regressor_iq.predict(iq_test).astype(int)

submission = pd.read_csv("submission_format.csv",
                         index_col=[0, 1, 2])


submission.total_cases = np.concatenate((sj_predictions, iq_predictions),axis=0)
submission.to_csv("SanJuan-RandomForest_Iquitos-RandomForest-lags-15-15.csv")
